In [ ]:
from stockfish import Stockfish
import numpy as np
import pandas as pd
import chess
import chess.pgn
import tqdm
import datetime
import timeit

# this is the path to the .exe of stockfish, not the stockfish python module
stockfish = Stockfish(path="stockfish_15.1_win_x64_avx2/stockfish-windows-2022-x86-64-avx2.exe")


In [ ]:
user = "settty"

rows=[]
with open(f'data/settty-chess-games-2022-12-15.pgn') as pgn:
    # tqdm is just for adding a progress bar
    # the for loop is needed because we need to call chess.pgn.read_game to get it to read each successive game in the pgn
    for i in tqdm.tqdm(range(50)):
        # will need to make the above line dynamic to the number of rowns in the pgn
        row = {}
        game = chess.pgn.read_game(pgn)
        # below line contains all the header info in a nested list. I have extracted it in the row['game_date']... lines below
        # if I need more header info, uncomment the line below and look for the right piece to extract
        #row['headers']=game.headers.__dict__
        row['game_date'] = game.headers["Date"]
        row['white'] = game.headers['White']
        row['black'] = game.headers['Black']
        # will use the termination statement to determine the winner. This will be easier than figuring out who won from the
        # result which would be difficult when playing friends
        row['result'] = game.headers['Termination']

        # get the user that won
        if row['white'] in row['result']:
            row['winner'] = row['white']
        elif row['black'] in row['result']:
            row['winner'] = row['black']
        else:
            row['winner'] = 'draw'

        # get the user color
        if row['white'] == user:
            row['user_color'] = 'white'
        else:
            row['user_color'] = 'black'

        # split into categories for counting winners
        if row['winner'] == user:
            row['winner_count'] = row['winner']
        elif row['winner'] == 'draw':
            row['winner_count'] = 'draw'
        else:
            row['winner_count'] = 'other'

        if 'EndTime' in game.headers:
            row['end_time'] = game.headers['EndTime']
        else:
            row['end_time'] = game.headers['EndDate']
        row['time_control'] = game.headers['TimeControl']
        row['white_elo'] = game.headers['WhiteElo']
        row['black_elo'] = game.headers['BlackElo']
        row['moves']=[x.uci() for x in game.mainline_moves()]
        rows.append(row)

#create a pd df of the extracted data
loaded_games = pd.DataFrame(rows)

#convert the date into a date object
loaded_games['game_date'] = pd.to_datetime(loaded_games['game_date'])

# need to fix this part if the file already exists and we're adding new id's. Should get the largest id and start from there
# assign a unique id to each game because chess.com doesn't supply one
loaded_games['game_id'] = np.arange(50, 100)

# explode the moves column. it is a column with each row containing a list
loaded_games = loaded_games.explode('moves')

#the line below adds all the moves into a single string of all the moves
loaded_games['moves_list'] = loaded_games.groupby('end_time')['moves'].apply(lambda x: (x + ' ').cumsum().str.strip())
loaded_games['moves_list'] = loaded_games['moves_list'].str.split()

# group each row by the end time and then add a counter for each row in the group. Then apply floor division to get the move
# number. a "move" is each player making a turn.
loaded_games['move_number'] = loaded_games.groupby('end_time').cumcount()//2
loaded_games['move_number_actual'] = loaded_games.groupby('end_time').cumcount()

In [ ]:
loaded_games.head()

In [ ]:
%%timeit -n 1 -r 10
rows = []

first_games = loaded_games.loc[(loaded_games['end_time'] == '16:44:56 PST')]

# the line below is for just a couple of games
for i, row in first_games.iterrows():
    new_row = {}
    new_row['end_time'] = row['end_time']
    stockfish.set_position(row['moves_list'])
    scores = stockfish.get_evaluation()
    new_row['move_number_actual'] = row['move_number_actual']
    new_row['score_type'] =(scores['type'])
    new_row['score'] = scores['value'] / 100
    rows.append(new_row)

score_df = pd.DataFrame(rows)
#scored_games = pd.merge(loaded_games, score_df, on=["end_time", "move_number_actual"])
first_games = first_games.merge(score_df, on=["end_time", "move_number_actual"])

scored_games['game_date'] = pd.to_datetime(scored_games['game_date'])

In [ ]:
%%timeit -n 1 -r 10
rows = []

first_games = loaded_games.loc[(loaded_games['end_time'] == '16:44:56 PST')]

# the line below is for just a couple of games
for i, row in first_games.iterrows():
    new_row = {}
    new_row['end_time'] = row['end_time']
    if (row['move_number_actual'] == 0):
        stockfish.set_position(row['moves_list'])
        scores = stockfish.get_evaluation()
    else:
        stockfish.make_moves_from_current_position([row['moves']])
        scores = stockfish.get_evaluation()
    new_row['move_number_actual'] = row['move_number_actual']
    new_row['score_type'] =(scores['type'])
    new_row['score'] = scores['value'] / 100
    rows.append(new_row)

score_df = pd.DataFrame(rows)
#scored_games = pd.merge(loaded_games, score_df, on=["end_time", "move_number_actual"])
first_games = first_games.merge(score_df, on=["end_time", "move_number_actual"])

scored_games['game_date'] = pd.to_datetime(scored_games['game_date'])

In [ ]:
scored_games